In [1]:
from __future__ import annotations

import pytz

from post_processing.utils.fpod_utils import csv_folder, pod2aplose, actual_data, meta_cut_aplose, feeding_buzz, txt_folder, add_utc
from post_processing.utils.core_utils import json2df

import logging
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas import Timestamp, to_datetime
from pandas.tseries import frequencies
from post_processing.dataclass.data_aplose import DataAplose
from post_processing.utils.core_utils import get_season, get_count
from osekit import setup_logging
from user_case.config import season_color
setup_logging(Path(r"C:\Users\dupontma2\Documents\Git\OSmOSE\OSmOSE_post_processing\src\post_processing\logging_config.yaml"), logging.ERROR)

### Load data
🐬 = input to modify \
Import your raw FPOD or CPOD data. All files for one site must be stored in the same folder and identified by their respective phases. \
You also need to import your metadata file.

In [2]:
pod_files = Path(r"U:\CA4") #Path to your data folder. 🐬
path = csv_folder(pod_files) #Process all your POD.csv files.

fb_files = Path(r"U:\fb_CA4")  #Path to your click details folder. 🐬
json = Path(r"C:\Users\fouinel\Downloads\deployment_calais.json") #Path to your metadata file. 🐬

print(path.head())
df_0 = path.dropna()

metadatax = json2df(json_path=json)
metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +  metadatax["campaign.name"].astype(str))

                      File          ChunkEnd DPM  Nall MinsOn deploy.name
0  CA4  POD2397 file01.CP3  14/05/2014 07:07   0     0      1  CA4_Phase1
1  CA4  POD2397 file01.CP3  14/05/2014 07:08   0     8      1  CA4_Phase1
2  CA4  POD2397 file01.CP3  14/05/2014 07:09   0     4      1  CA4_Phase1
3  CA4  POD2397 file01.CP3  14/05/2014 07:10   0   251      1  CA4_Phase1
4  CA4  POD2397 file01.CP3  14/05/2014 07:11   0  4095      1  CA4_Phase1


In [ ]:
d_beg_end = actual_data(df_0, metadatax) #Extract the beginning and end of recording for every phase.

In [3]:
df_1 = df_0[df_0["DPM"] !="0" ] #Remove the 0 to lighten the APLOSE file.

### APLOSE format
#### *C-POD*
Use cpod2aplose if you are managing C-POD data.

In [4]:
df_aplose = pod2aplose(df_1, pytz.utc, "CA4", "Marsouin", "CPOD") #Precise site name, species and instrument. 🐬
print(df_aplose.head())

  dataset                      filename  start_time  end_time  \
0     CA4  2014-05-16T02:30:00.000+0000           0        60   
1     CA4  2014-05-17T03:52:00.000+0000           0        60   
2     CA4  2014-05-17T04:47:00.000+0000           0        60   
3     CA4  2014-05-19T17:06:00.000+0000           0        60   
4     CA4  2014-05-20T11:07:00.000+0000           0        60   

   start_frequency  end_frequency annotation annotator  \
0                0              0   Marsouin      CPOD   
1                0              0   Marsouin      CPOD   
2                0              0   Marsouin      CPOD   
3                0              0   Marsouin      CPOD   
4                0              0   Marsouin      CPOD   

                 start_datetime                  end_datetime  is_box  \
0  2014-05-16T02:30:00.000+0000  2014-05-16T02:31:00.000+0000       0   
1  2014-05-17T03:52:00.000+0000  2014-05-17T03:53:00.000+0000       0   
2  2014-05-17T04:47:00.000+0000  2014-05-

### Clean your dataset
Remove useless lines, recorder outside the instrument submersion. Export your file to the aplose format. You can change the name of the file to match the project you are working on.

In [5]:
cleared = meta_cut_aplose(df_aplose, metadatax)

In [7]:
cleared = cleared.drop_duplicates(subset=['start_datetime'], keep="last")

### Feeding buzzes processing
Use "Dauphin", Marsouin" or "Commerson" to get different ICI processing.

In [ ]:
fb_all = txt_folder(fb_files) #Read all your FB.txt files.
fb_all = feeding_buzz(fb_all, "Marsouin") #Categorize the minutes (positive or not to FB detection). 🐬
add_utc(fb_all, ["start_datetime"], "min")

In [ ]:
dpm_fb = cleared.merge(fb_all[["start_datetime", "Foraging"]], on=["start_datetime"], how="left") #Merge DPM and FB dataframes

### Export

In [ ]:
d_beg_end.to_csv(r"U:\Deb_Fin_CA4.csv", index=False) #Export the new file. 🐬
dpm_fb.to_csv(r"U:\APLOSE_CA4_pos.csv", index=False) #Name your file. 🐬

### Explore
First visualization of the data

In [ ]:
data = DataAplose(dpm_fb)

In [ ]:
bin_size = frequencies.to_offset("1d")
ticks = frequencies.to_offset("6BMS")
fmt = "%b %y"

In [ ]:
data.lat = 50.973333 #CA4: 51.00035 ; Walde: 50.973333 ; A: -49.38765 ; B: -49.424733 ; C: -49.4677 ; D: -49.47175
data.lon = 1.8117 #CA4: 1.879667 ; Walde: 1.8117 ; A: 69.9449 ; B: 69.932383 ; C: 70.081067 ; D: 69.836617

#### Reshape the data
Set beginning and end of the chosen window.

In [ ]:
data.df["end_datetime"] = to_datetime(data.df["end_datetime"])
data2 = data.reshape(begin=Timestamp("2013 11 01"), end=Timestamp("2025 08 01"))
tz = pytz.timezone("UTC")
data2.change_tz(tz)

#### Heatmap

In [ ]:
fig, ax = plt.subplots(1, 1)
ax = data2.set_ax(ax=ax, x_ticks_res=ticks, date_format=fmt)
data2.plot(
    mode="heatmap",
    annotator=data2.annotators[0],
    label=data2.labels[0],
    ax=ax,
    bin_size=bin_size,
    show_rise_set=True,
)
plt.tight_layout()
plt.show()

#### Detections over time

In [ ]:
fig2, ax = plt.subplots(1, 1)
ax = data2.set_ax(ax=ax, x_ticks_res=ticks, date_format=fmt)
data2.plot(
    mode="scatter",
    annotator=data2.annotators[0],
    label=data2.labels[0],
    ax=ax,
    show_rise_set=True,
)
plt.tight_layout()
plt.show()

#### DPM per day

In [ ]:
df_filtered = data2.filter_df(data2.annotators[0], data2.labels[0])
df_counts = get_count(df_filtered, bin_size)

df_counts["Season"] = df_counts.index.to_series().apply(lambda x: get_season(x)[0])
df_counts["colors"] = df_counts["Season"].map(season_color).fillna("gray")

fig3, ax = plt.subplots(1, 1)
ax = data2.set_ax(ax=ax, x_ticks_res=ticks, date_format=fmt)
data2.plot(
    mode="histogram",
    annotator=data2.annotators[0],
    label=data2.labels[0],
    color=df_counts["colors"].tolist(),
    ax=ax,
    bin_size=bin_size,
    legend=True,
)
ax.set_ylim(0, 200)
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=10))
plt.tight_layout()
plt.show()